# Перевод датасета EmpathicIntents
https://github.com/anuradha1992/EmpatheticIntents.git

In [1]:
import sys
import os.path as osp
import os

current_dir = os.getcwd()

parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)
sys.path.append(current_dir + '/notebooks')
sys.path.append(current_dir)
sys.path.append(current_dir + '/data')

In [16]:
import json
from pathlib import Path
import re

from datasets import Dataset
from omegaconf import OmegaConf
from copy import deepcopy
from collections import defaultdict
import pandas as pd
from src.core.translate import Translator
from src.utils.schemas import GeneralTranslationResultSchema

вспомогательные функции

In [688]:
regex_check_russian = re.compile(r'^[а-яА-ЯёЁ0-9\s,.!?;:\-—()«»„“"\'’]+$')
def read_json(path):
    return json.load(Path(path).open())

def read_file(path: str):
	return Path(path).open().read()

def save_json(obj, path):
    json.dump(obj, Path(path).open("w", encoding="utf-8"), indent=4, ensure_ascii=False)

def is_russian_text(text):
    # Разрешаем:
    # - русские буквы (включая ёЁ)
    # - цифры (0-9)
    # - пробелы, переносы строк (\s)
    # - стандартные знаки препинания: ,.!?;:()
    # — длинное тире (—), дефис (-)
    # - кавычки («»„““”"')
    # - апостроф (’') для слов типа "селёдочка"
    return bool(re.fullmatch(regex_check_russian, text))

In [10]:
from glob import glob
files = glob('/Users/n.valieva/Desktop/empathy_dataset_transfer/EmpatheticIntents/datasets/empatheticdialogues_annotated/*.csv')
mapping = {Path(f).stem: pd.read_csv(f) for f in files}

в датасете содержится 32 файла с аннотациями:

In [582]:
print(sorted(mapping.keys()))

['afraid', 'angry', 'annoyed', 'anticipating', 'anxious', 'apprehensive', 'ashamed', 'caring', 'confident', 'content', 'devastated', 'disappointed', 'disgusted', 'embarrassed', 'excited', 'faithful', 'furious', 'grateful', 'guilty', 'hopeful', 'impressed', 'jealous', 'joyful', 'lonely', 'nostalgic', 'prepared', 'proud', 'sad', 'sentimental', 'surprised', 'terrified', 'trusting']


In [164]:
len(mapping)

32

In [15]:
mapping['afraid'].head()

,Dialog_ID,Type,Actor,Text,Label
0,hit:4821_conv:9643,situation,none,When my car caught on fire while driving,devastated
1,hit:4821_conv:9643,utterance,speaker,I had my car catch on fire while driving!,devastated
2,hit:4821_conv:9643,utterance,listener,"Oh my gosh, what did you do?",questioning
3,hit:4821_conv:9643,utterance,speaker,Pulled over as fast as I could and jumped out....,afraid
4,hit:4821_conv:9643,utterance,listener,"I bet, glad you are ok",encouraging


In [21]:
len(mapping['afraid'])

4238

соберем данные в нужном формате для перевода, для начала переведем просто все тексты из диалога

In [20]:
datasets = defaultdict(list)
for i, (fname, df) in enumerate(mapping.items()):
    ids = df['Dialog_ID'].unique().tolist()
    for id in ids:
        texts = df[df['Dialog_ID'] == id]['Text'].tolist()
        for j in range(len(texts)):
            datasets[fname].append({"id": f"{id}_{j}", "text": texts[j]})

In [22]:
len(datasets['afraid'])

4238

In [23]:
datasets['afraid'][0]

{'id': 'hit:4821_conv:9643_0',
 'text': 'When my car caught on fire while driving'}

In [24]:
for fname, data in datasets.items():
    save_json(data, f"{fname}.json")

## Перевод одного файла

In [685]:
name = "afraid"

In [686]:
dataset = read_json(f'{name}.json')

In [687]:
len(dataset)

4238

подгружаем конфиг для перевода, в этом случае используем батч из 32 текстов

In [ ]:
config = OmegaConf.load('../configs/conf.yaml')
general_translation_config = config.general_translation
general_translation_config.batch_result_dir = f"batches_res_general_translation_{name}"
general_translation_config.batches = f"batches_general_translation_{name}"
general_translation_config.batch_size = 32

In [572]:
general_translator = Translator(
    system_message=read_file(general_translation_config.prompt_path), 
    model_config=read_json(general_translation_config.model_config_path), 
    example_data=read_json(general_translation_config.filepath_examples), 
    batch_size=general_translation_config.batch_size,
    batch_result_dir=general_translation_config.batch_result_dir,
    batch_dir=general_translation_config.batches,
    model_type="openai"
)

In [ ]:
print(read_json(general_translation_config.model_config_path))

In [ ]:
general_input_dataset = Dataset.from_list(dataset)

translation_result = general_translator.translate(general_input_dataset, GeneralTranslationResultSchema)

In [ ]:
save_json(translation_result, f"translation_result_empathic_intents_{name}.json")

In [577]:
translation_result = read_json("translation_result_empathic_intents_apprehensive_qwen.json")

смотрим те случаи, когда в тексте есть какие-то иноязычные символы и корректируем их перевод с помощью `gpt-4o`

In [ ]:
to_correct = []
for item in translation_result:
    if not is_russian_text(item['text_rus']):
        to_correct.append(item)
        print(item['text_rus'])
        print('***************'*10)

In [546]:
len(to_correct)

165

In [547]:
dataset_ids = set([item['id'] for item in dataset])
tr_ids = set([item['id'] for item in translation_result])

собираем те тексты, которые не перевелись

In [548]:
more = dataset_ids - tr_ids

In [549]:
len(more)

68

смотрим те случаи, когда у нас в переводе появились тексты, которых нет в датасете. В таких примерах можно вручную изменить `id`, чтобы они совпадали с тем, что есть в датасете.

In [551]:
tr_ids - dataset_ids

set()

In [553]:
for item in more:
    for d in dataset:
        if d['id'] == item:
            to_correct.append(d)

In [ ]:
len(to_correct), len(more)

In [558]:
for item in to_correct:
    if 'text_rus' in item:
        del item['text_rus']

In [ ]:
to_correct_ids = set([i['id'] for i in to_correct])

In [ ]:
for i, item in enumerate(translation_result):
    if item['id'] in to_correct_ids:
        translation_result.remove(item)

In [ ]:
len(translation_result), len(dataset)

In [ ]:
len(dataset) - len(translation_result) == len(to_correct)

In [ ]:
save_json(translation_result, f'translation_result_empathic_intents_{name}.json')

In [ ]:
config = OmegaConf.load('../configs/conf.yaml')
general_translation_config = config.general_translation
general_translation_config.batch_result_dir = f"batches_res_general_translation_{name}"
general_translation_config.batches = f"batches_general_translation_{name}"
general_translation_config.batch_size = 64
model_config = read_json(general_translation_config.model_config_path)
model_config['model'] = 'gpt-4o'

In [ ]:
general_translator = Translator(
    system_message=read_file(general_translation_config.prompt_path), 
    model_config=model_config, 
    example_data=read_json(general_translation_config.filepath_examples), 
    batch_size=general_translation_config.batch_size,
    batch_result_dir=general_translation_config.batch_result_dir,
    batch_dir=general_translation_config.batches,
    model_type="openai"
)

In [ ]:
general_input_dataset = Dataset.from_list(to_correct)

translation_result_corrected = general_translator.translate(general_input_dataset, GeneralTranslationResultSchema)

In [ ]:
all_translation = translation_result_corrected | translation_result

In [ ]:
save_json(all_translation, f'translation_result_empathic_intents_{name}_all.json')

In [ ]:
translated = read_json(f'translation_result_empathic_intents_{name}_all.json')

In [ ]:
len(translated), len(dataset)

(3262, 3262)

In [ ]:
set([i['id'] for i in translated]) - set([i['id'] for i in dataset])

set()

In [ ]:
set([i['id'] for i in dataset]) - set([i['id'] for i in translated])

set()

## Собираем переводы всех файлов

In [ ]:
all_names = ['afraid', 'angry', 'annoyed', 'anticipating', 'anxious', 'apprehensive', 'ashamed', 
'caring', 'confident', 'content', 'devastated', 'disappointed', 'disgusted', 'embarrassed', 
'excited', 'faithful', 'furious', 'grateful', 'guilty', 'hopeful', 'impressed', 'jealous', 
'joyful', 'lonely', 'nostalgic', 'prepared', 'proud', 'sad', 'sentimental', 'surprised', 'terrified', 'trusting']


словарь с переводами каждого из файлов

In [597]:
files = {
    'afraid': 'translation_result_empathic_intents_afraid_qwen.json',
    'angry': 'translation_result_empathic_intents_angry_removing_errors.json',
    'annoyed': 'translation_result_empathic_intents_annoyed_qwen.json',
    'anticipating': 'translation_result_empathic_intents_anticipating_qwen.json',
    'anxious': 'translation_result_empathic_intents_anxious_qwen.json',
    'apprehensive': 'translation_result_empathic_intents_apprehensive_qwen.json',
    'ashamed': 'translation_result_empathic_intents_ashamed_qwen.json',
    'caring': 'translation_result_empathic_intents_caring_qwen.json',
    'confident': 'translation_result_empathic_intents_confident_qwen.json',
    'content': 'translation_result_empathic_intents_content_qwen.json',
    'devastated': 'translation_result_empathic_intents_devastated_qwen.json',
    'disappointed': 'translation_result_empathic_intents_disappointed_qwen.json',
    'disgusted': 'translation_result_empathic_intents_disgusted_qwen.json',
    'embarrassed': 'translation_result_empathic_intents_embarrassed_qwen.json',
    'excited': 'translation_result_empathic_intents_excited_qwen.json',
    'faithful': 'translation_result_empathic_intents_faithful_qwen.json',
    'furious': 'translation_result_empathic_intents_furious_qwen.json',
    'grateful': 'translation_result_empathic_intents_grateful_qwen.json',
    'guilty': 'translation_result_empathic_intents_guilty_qwen.json',
    'hopeful': 'translation_result_empathic_intents_hopeful_qwen.json',
    'impressed': 'translation_result_empathic_intents_impressed_qwen.json',
    'jealous': 'translation_result_empathic_intents_jealous_qwen.json',
    'joyful': 'translation_result_empathic_intents_joyful_qwen.json',
    'lonely': 'translation_result_empathic_intents_lonely_qwen.json',
    'nostalgic': 'translation_result_empathic_intents_nostalgic_qwen.json',
    'prepared': 'translation_result_empathic_intents_prepared_qwen.json',
    'proud': 'translation_result_empathic_intents_proud_qwen.json',
    'sad': 'translation_result_empathic_intents_sad_qwen.json',
    'sentimental': 'translation_result_empathic_intents_sentimental_qwen.json',
    'surprised': 'translation_result_empathic_intents_surprised_qwen.json',
    'terrified': 'translation_result_empathic_intents_terrified_qwen.json',
    'trusting': 'translation_result_empathic_intents_trusting_qwen.json',
}

In [689]:
for name in files:
    if not Path(files[name]).exists():
        print(name)

In [ ]:
ACTOR = {
    'none': "никто",
    'speaker': "рассказщик",
    'listener': 'слушатель'
}

TYPE = {
    'situation': 'ситуация',
    'utterance': 'высказывание'
}

ALL_LABELS = {
  "surprised": "удивлённый",  
  "sentimental": "сентиментальный",  
  "content": "довольный",  
  "trusting": "доверчивый",  
  "disappointed": "разочарованный",  
  "questioning": "сомневающийся",  
  "sad": "грустный",  
  "agreeing": "соглашающийся",  
  "joyful": "радостный",  
  "encouraging": "ободряющий",  
  "annoyed": "раздражённый",  
  "terrified": "напуганный",  
  "consoling": "утешающий",  
  "wishing": "желающий",  
  "acknowledging": "признающий",  
  "neutral": "нейтральный",  
  "nostalgic": "ностальгирующий",  
  "lonely": "одинокий",  
  "anxious": "тревожный",  
  "embarrassed": "смущённый",  
  "proud": "гордый",  
  "guilty": "виноватый",  
  "prepared": "готовый",  
  "furious": "разъярённый",  
  "caring": "заботливый",  
  "confident": "уверенный",  
  "jealous": "ревнивый",  
  "hopeful": "надеющийся",  
  "impressed": "впечатлённый",  
  "disgusted": "отвращённый",  
  "grateful": "благодарный",  
  "excited": "взволнованный",  
  "faithful": "верный",  
  "apprehensive": "опасающийся",  
  "suggesting": "предлагающий",  
  "angry": "злой",  
  "devastated": "опустошённый",  
  "afraid": "испуганный",  
  "anticipating": "ожидающий",  
  "sympathizing": "сочувствующий",  
  "ashamed": "пристыженный"  
}

собираем файл, аналогичный англискому, на русском языке

In [ ]:
all_data = {}
for name in files:
    fname = Path(f'/Users/n.valieva/Desktop/empathy_dataset_transfer/EmpatheticIntents/datasets/empatheticdialogues_annotated/{name}.csv') 
    df = pd.read_csv(fname)
    tr = read_json(files[name])
    all_ids = df.Dialog_ID.unique().tolist()
    all_diag = {}
    all_data[name] = defaultdict(list)

    for i, item in enumerate(tr):
        upper_id = '_'.join(item['id'].split('_')[:2]).strip()
        tr[i]['upper_id'] = upper_id
    
    for id in all_ids:
        items = [item for item in tr if item['upper_id'] == id]
        sorted_ids = sorted(items, key=lambda x: int(x['id'].split('_')[-1]))
        all_diag[id] = sorted_ids
    
    for id in all_diag:
        for item in all_diag[id]:
            all_data[name]['Dialog_ID'].append(id)
            all_data[name]['Text'].append(item['text_rus'])
    df_ru = pd.DataFrame.from_dict(all_data[name])
    assert (df_ru["Dialog_ID"] == df["Dialog_ID"]).all(),  "Dialog_ID mismatch!"
    df_ru['Text_eng'] = df["Text"]
    df_ru["Type"] = df["Type"].apply(lambda x: TYPE[x])
    df_ru["Actor"] = df["Actor"].apply(lambda x: ACTOR[x])
    df_ru["Label"] = df["Label"].apply(lambda x: ALL_LABELS[x])
    df_ru.to_csv(f"empathic_intents_rus/{name}_rus.csv")

In [683]:
df_ru

,Dialog_ID,Text,Text_eng,Type,Actor,Label
0,hit:4375_conv:8750,В прошлые выходные я оставил свою квартиру дру...,I left my apartment to my friend last weekend....,ситуация,никто,доверчивый
1,hit:4375_conv:8750,Я уехал в другую страну на 2 месяца и оставил ...,I am away in another country for 2 months and ...,высказывание,рассказщик,доверчивый
2,hit:4375_conv:8750,"Вау, должно быть страшно доверить все свои вещ...","Wow, that must be scary to have to trust someo...",высказывание,слушатель,сомневающийся
3,hit:4375_conv:8750,"Я в отпуске на Багамах! Я уверен, что они наде...",I'm on vacation in the Bahamas! I feel they ar...,высказывание,рассказщик,доверчивый
4,hit:4375_conv:8750,Двухмесячный отпуск звучит потрясающе! Я думал...,A two month vacation sounds amazing! I figured...,высказывание,слушатель,сомневающийся
...,...,...,...,...,...,...
3521,hit:906_conv:1812,"Когда я катался на катапульте, это было ужасно!","When I went on the slingshot ride, it was horr...",ситуация,никто,напуганный
3522,hit:906_conv:1812,"Я катался на этой аттракционе на пляже, которы...",I went on this ride at the beach called the Sl...,высказывание,рассказщик,доверчивый
3523,hit:906_conv:1812,"Ого, я видел один из таких!","Omg, I have seen one of those!",высказывание,слушатель,напуганный
3524,hit:906_conv:1812,"Это было реально так страшно, я еле удержался,...","It was seriously so scary, I almost pooped mys...",высказывание,рассказщик,напуганный
